In [2]:
import tweepy
import configparser
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle
import nltk
import string
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer

In [79]:
train_label_file = open('project-data/train.label.txt', 'r')
train_labels = train_label_file.readlines()
train_labels = [label.strip('\n') for label in train_labels]
    
dev_label_file = open('project-data/dev.label.txt', 'r')
dev_labels = dev_label_file.readlines()
dev_labels = [label.strip('\n') for label in dev_labels]

In [80]:
# open train text file
f = open(f'./tweet_text.pckl','rb')
train_data = pickle.load(f)
f.close()


# open dev text file
f = open(f'./dev_tweet_text.pckl','rb')
dev_data = pickle.load(f)
f.close()

In [81]:
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mention
    text = re.sub(r'#','',text) # remove the hashtag symbol
    text = re.sub(r'https?:\/\/\S+', '',text) #remove hyperlink
    text = re.sub(r'\n','',text) # remove \n 
    return text

for i in range(len(train_data)):
    for j in range(len(train_data[i])):
        train_data[i][j] = clean_text(train_data[i][j])
        
for i in range(len(dev_data)):
    for j in range(len(dev_data[i])):
        dev_data[i][j] = clean_text(dev_data[i][j])

In [82]:
# merge source tweeet and reply tweet together for train data
train_merge_events=[]
for event in train_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    train_merge_events.append(merge)
    
    
# merge source tweeet and reply tweet together for dev data
dev_merge_events=[]
for event in dev_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    dev_merge_events.append(merge)

In [52]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [83]:
y_train = [1 if x== 'nonrumour' else 0 for x in train_labels]
y_dev = [1 if x== 'nonrumour' else 0 for x in dev_labels]

In [84]:
tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(train_merge_events)
x_train = tokenizer.texts_to_sequences(train_merge_events)
x_dev = tokenizer.texts_to_sequences(dev_merge_events)
y_train = np.array(y_train)
y_dev = np.array(y_dev)

In [85]:
max_train = 0
for each in x_train:
    l = len(each)
    if l > max_train:
        max_train = l

max_dev = 0
for each in x_dev:
    l = len(each)
    if l > max_dev:
        max_dev = l

if max_train > max_dev:
    maxlen = max_train
else:
    maxlen = max_dev

In [86]:
xseq_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
xseq_dev = pad_sequences(x_dev, padding='post', maxlen=maxlen)

In [89]:
from keras.layers import LSTM
from keras.models import Sequential
from keras import layers

In [90]:
vocab_size = xseq_train.shape[1]
embedding_dim = 10
vocab_size

2688

In [91]:
model3 = Sequential(name="lstm")
model3.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model3.add(LSTM(10))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model3.summary()

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2688, 10)          26880     
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 27,731
Trainable params: 27,731
Non-trainable params: 0
_________________________________________________________________


In [92]:
model3.fit(xseq_train, y_train, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev), batch_size=10)

loss, accuracy = model3.evaluate(xseq_dev, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20


InvalidArgumentError:  indices[4,17] = 4278 is not in [0, 2688)
	 [[node lstm/embedding_6/embedding_lookup (defined at \LEHOAN~1\AppData\Local\Temp/ipykernel_18372/479889254.py:1) ]] [Op:__inference_train_function_22154]

Errors may have originated from an input operation.
Input Source operations connected to node lstm/embedding_6/embedding_lookup:
 lstm/embedding_6/embedding_lookup/20956 (defined at \LE HOANG THY\anaconda3\envs\CV\lib\contextlib.py:113)

Function call stack:
train_function


In [ ]:
## extract embedding to see the similiarity between tweets???